### 연구 목적: Indoor moving patterns들을 이용하여 revisit intention을 예측.
<pre>


</pre>
__어떤 유저의 Revisit intention을 예측하는 supervised learning 모델에 이용될 간단한 Feature들:__
1. User가 찍힌 로그 총 개수: num_logs
2. 한 User가 와이파이에 잡힌 총 시간: total_dwell_time
3. dwell_time > 10인 indoor area 개수: num_area_10
4. indoor 로그 중 dwell_time > 10인 확률: prob_dwell_10
5. 전체 로그 중 deny=True인 확률: prob_deny



__여기서의 초점__
1. 한 유저에 대한 log가 여러 개인데, 일단 aggregate하여 유저당 하나의 row로 나타낼 수 있도록 함)
2. 가지고 있는 데이터셋의 revisit_count는 같은 날 두 번 방문하면 올라가지 않는다 - 유저별 max(revisit_count) > 0 이면 yes, 아니면 no로 레이블링
3. 정확한 트레이닝은, 특정 시점 이전의 데이터를 이용하여 특정 시점 이후에 고객이 방문했는지 안 했는지 조사해야 하지만, 귀찮으므로 일단은 지금까지 전체 데이터를 가지고 revisit count를 예측하는 모델로 구성

__데이터__
1. 781번 매장 데이터 (10%의 유저를 테스트로 이용)

In [1]:
### import libraries
import pandas as pd
import datetime
import numpy as np

In [2]:
df = pd.read_pickle("../data/781/781.p")

In [94]:
# len(df['device_id'].unique())
# uniqIDs = df['device_id'].unique()
# columns = ['num_logs', 'total_dwell_time', 'num_area_10', 'prob_dwell_10', 'prob_deny']

In [95]:
f1, f2, f3, f4 ,f5, label = feature_generator(df)

In [96]:
df2 = df_generator(df_toy, f1, f2, f3, f4, f5, label)

In [101]:
# Check methods works right by comparing with test dataframe
df2.loc[['0d4fd55bb363bf6f6f7f8b3342cd0467','15493306b1ea43056a690005e664d346','839d202915edc20699f08f733d46f030','f076ab559ef130c78ab7123d3807c1fc','f6258edf9145d1c0404e6f3d7a27a29d'],:]

,num_logs,total_dwell_time,num_area_10,prob_dwell_10,prob_deny,revisit_count
0d4fd55bb363bf6f6f7f8b3342cd0467,3489.0,8111450.0,1725.0,0.542112,0.980797,68.0
15493306b1ea43056a690005e664d346,3241.0,689201.0,NaN,NaN,0.971305,4.0
839d202915edc20699f08f733d46f030,2674.0,1042562.0,NaN,NaN,0.945026,3.0
f076ab559ef130c78ab7123d3807c1fc,2235.0,974213.0,NaN,NaN,0.925280,NaN
f6258edf9145d1c0404e6f3d7a27a29d,13196.0,33024302.0,7449.0,0.607140,0.997272,81.0


### Test:  Small dataframe (2 people)

In [8]:
criterion = df['device_id'].map(lambda x: x.startswith('f6258edf9145d1c0404e6f3d7a27a29d') or x.startswith('0d4fd55bb363bf6f6f7f8b3342cd0467') or x.startswith('f076ab559ef130c78ab7123d3807c1fc') or x.startswith('839d202915edc20699f08f733d46f030') or x.startswith('15493306b1ea43056a690005e664d346'))
df_toy = df[criterion]

In [9]:
df_toy.shape

(24835, 8)

In [10]:
df_toy.columns

Index(['area', 'deny', 'device_id', 'dwell_time', 'key', 'revisit_count',
       'revisit_period', 'ts'],
      dtype='object')

In [11]:
df_toy['device_id'].unique()

array(['f076ab559ef130c78ab7123d3807c1fc',
       '839d202915edc20699f08f733d46f030',
       'f6258edf9145d1c0404e6f3d7a27a29d',
       '15493306b1ea43056a690005e664d346',
       '0d4fd55bb363bf6f6f7f8b3342cd0467'], dtype=object)

In [44]:
### F1: 로그 총 개수
# df_toy11 = df_toy['device_id'].value_counts()
f1 = df_toy.groupby(['device_id'])['ts'].count()
print(f1)

device_id
0d4fd55bb363bf6f6f7f8b3342cd0467     3489
15493306b1ea43056a690005e664d346     3241
839d202915edc20699f08f733d46f030     2674
f076ab559ef130c78ab7123d3807c1fc     2235
f6258edf9145d1c0404e6f3d7a27a29d    13196
Name: ts, dtype: int64


In [14]:
### F2: 와이파이에 잡힌 총 시간
f2 = df_toy.groupby(['device_id'])['dwell_time'].sum()
f2

device_id
0d4fd55bb363bf6f6f7f8b3342cd0467     8111450
15493306b1ea43056a690005e664d346      689201
839d202915edc20699f08f733d46f030     1042562
f076ab559ef130c78ab7123d3807c1fc      974213
f6258edf9145d1c0404e6f3d7a27a29d    33024302
Name: dwell_time, dtype: int64

In [15]:
### F3: dwell_time > 100인 indoor area 개수
df_toy_indoor = df_toy.loc[df_toy['area']!='out']
df_toy_indoor2 = df_toy_indoor.loc[df_toy_indoor['dwell_time']>100]
f3 = df_toy_indoor2.groupby(['device_id'])['area'].count()
print(f3)

device_id
0d4fd55bb363bf6f6f7f8b3342cd0467    1725
f6258edf9145d1c0404e6f3d7a27a29d    7449
Name: area, dtype: int64


In [16]:
### F4: indoor 로그 중 dwell_time > 10인 확률
f3_2 = df_toy_indoor.groupby(['device_id'])['area'].count()
f4 = f3.div(f3_2)
print (f4)


device_id
0d4fd55bb363bf6f6f7f8b3342cd0467    0.542112
15493306b1ea43056a690005e664d346         NaN
839d202915edc20699f08f733d46f030         NaN
f6258edf9145d1c0404e6f3d7a27a29d    0.607140
Name: area, dtype: float64


In [17]:
### F5: deny = True일 확률
# df_toy_true = df_toy.loc[df_toy['deny']==True]
# df_toy.groupby(['device_id'])['deny'].count()
# print(df_toy.groupby(['device_id', 'deny']).count())
a = df_toy.groupby(['device_id']).deny.count()
b = df_toy['device_id'].value_counts()
f5 = a.div(b)
# print (a, b)
print (f5)

0d4fd55bb363bf6f6f7f8b3342cd0467    0.980797
15493306b1ea43056a690005e664d346    0.971305
839d202915edc20699f08f733d46f030    0.945026
f076ab559ef130c78ab7123d3807c1fc    0.925280
f6258edf9145d1c0404e6f3d7a27a29d    0.997272
dtype: float64


In [18]:
# df_toy2['device_id'] = 
np.sort(df_toy['device_id'].unique())

array(['0d4fd55bb363bf6f6f7f8b3342cd0467',
       '15493306b1ea43056a690005e664d346',
       '839d202915edc20699f08f733d46f030',
       'f076ab559ef130c78ab7123d3807c1fc',
       'f6258edf9145d1c0404e6f3d7a27a29d'], dtype=object)

In [35]:
device_ids_toy = np.sort(df_toy['device_id'].unique())

In [36]:
df_toy2 = pd.DataFrame(columns=columns, index=device_ids_toy)

In [37]:
# df_toy2['device_id'] = np.sort(df_toy['device_id'].unique())

In [47]:
df_toy2

,num_logs,total_dwell_time,num_area_10,prob_dwell_10,prob_deny
0d4fd55bb363bf6f6f7f8b3342cd0467,NaN,NaN,NaN,NaN,NaN
15493306b1ea43056a690005e664d346,NaN,NaN,NaN,NaN,NaN
839d202915edc20699f08f733d46f030,NaN,NaN,NaN,NaN,NaN
f076ab559ef130c78ab7123d3807c1fc,NaN,NaN,NaN,NaN,NaN
f6258edf9145d1c0404e6f3d7a27a29d,NaN,NaN,NaN,NaN,NaN


In [56]:
df_toy2["num_logs"] = f1
df_toy2["total_dwell_time"] = f2
df_toy2["num_area_10"] = f3
df_toy2["prob_dwell_10"] = f4
df_toy2["prob_deny"] = f5

In [57]:
df_toy2

,num_logs,total_dwell_time,num_area_10,prob_dwell_10,prob_deny
0d4fd55bb363bf6f6f7f8b3342cd0467,3489,8111450,1725.0,0.542112,0.980797
15493306b1ea43056a690005e664d346,3241,689201,NaN,NaN,0.971305
839d202915edc20699f08f733d46f030,2674,1042562,NaN,NaN,0.945026
f076ab559ef130c78ab7123d3807c1fc,2235,974213,NaN,NaN,0.925280
f6258edf9145d1c0404e6f3d7a27a29d,13196,33024302,7449.0,0.607140,0.997272


In [61]:
##### 삽질한 코드들#####
# pd.concat([df_toy2, f4], axis=1)
# df_toy2.merge(f1, on='device_id', how='left')
# df_toy2['total_dwell_time'] = df_toy.groupby(['device_id'])['dwell_time'].sum()
# df_toy2 = pd.concat([df_toy2, df_toy12], axis=1)
# df_toy12f = df_toy12.to_frame()
# df_toy12.get

In [66]:
label_toy = df_toy.groupby(['device_id'])['revisit_count'].max()
label_toy

device_id
0d4fd55bb363bf6f6f7f8b3342cd0467    68.0
15493306b1ea43056a690005e664d346     4.0
839d202915edc20699f08f733d46f030     3.0
f076ab559ef130c78ab7123d3807c1fc     NaN
f6258edf9145d1c0404e6f3d7a27a29d    81.0
Name: revisit_count, dtype: float64

In [69]:
pd.concat([df_toy2, label_toy], axis=1)

,num_logs,total_dwell_time,num_area_10,prob_dwell_10,prob_deny,revisit_count
device_id,,,,,,
0d4fd55bb363bf6f6f7f8b3342cd0467,3489,8111450,1725.0,0.542112,0.980797,68.0
15493306b1ea43056a690005e664d346,3241,689201,NaN,NaN,0.971305,4.0
839d202915edc20699f08f733d46f030,2674,1042562,NaN,NaN,0.945026,3.0
f076ab559ef130c78ab7123d3807c1fc,2235,974213,NaN,NaN,0.925280,NaN
f6258edf9145d1c0404e6f3d7a27a29d,13196,33024302,7449.0,0.607140,0.997272,81.0


### 정리하기

In [ ]:
criterion = df['device_id'].map(lambda x: x.startswith('f6258edf9145d1c0404e6f3d7a27a29d') or x.startswith('0d4fd55bb363bf6f6f7f8b3342cd0467') or x.startswith('f076ab559ef130c78ab7123d3807c1fc') or x.startswith('839d202915edc20699f08f733d46f030') or x.startswith('15493306b1ea43056a690005e664d346'))
df_toy = df[criterion]

In [82]:
def feature_generator(df_toy):
    ### F1: 로그 총 개수
    f1 = df_toy.groupby(['device_id'])['ts'].count()
#     print(f1)
    
    ### F2: 와이파이에 잡힌 총 시간
    f2 = df_toy.groupby(['device_id'])['dwell_time'].sum()
#     print(f2)

    ### F3: dwell_time > 100인 indoor area 개수
    df_toy_indoor = df_toy.loc[df_toy['area']!='out']
    df_toy_indoor2 = df_toy_indoor.loc[df_toy_indoor['dwell_time']>100]
    f3 = df_toy_indoor2.groupby(['device_id'])['area'].count()
#     print(f3)

    ### F4: indoor 로그 중 dwell_time > 10인 확률
    f3_2 = df_toy_indoor.groupby(['device_id'])['area'].count()
    f4 = f3.div(f3_2)
#     print (f4)
    
    ### F5: deny = True일 확률
    a = df_toy.groupby(['device_id']).deny.count()
    b = df_toy['device_id'].value_counts()
    f5 = a.div(b)
#     print (f5)

    ### Label: Maximum revisit count from the log
    label_toy = df_toy.groupby(['device_id'])['revisit_count'].max()
#     print(label_toy)

    return f1, f2, f3, f4, f5, label_toy

In [84]:
f1, f2, f3, f4 ,f5, label_toy = feature_generator(df_toy)

In [90]:
def df_generator(df_toy, f1, f2, f3, f4, f5, label_toy):
    columns = ['num_logs', 'total_dwell_time', 'num_area_10', 'prob_dwell_10', 'prob_deny']
    
    # feature들과의 index의 통일을 위해 np.sort를 이용.
    device_ids_toy = np.sort(df_toy['device_id'].unique())       
    df_toy2 = pd.DataFrame(columns=columns, index=device_ids_toy)
    
    # feature를 df에 삽입
    df_toy2["num_logs"] = f1          
    df_toy2["total_dwell_time"] = f2
    df_toy2["num_area_10"] = f3
    df_toy2["prob_dwell_10"] = f4
    df_toy2["prob_deny"] = f5
    
    # label을 df에 합침
    df_toy2 = pd.concat([df_toy2, label_toy], axis=1)   
    
    # machine learning에 바로 이용될 dataframe을 리턴
    return df_toy2
    

In [92]:
df_toy2 = df_generator(df_toy, f1, f2, f3, f4, f5, label_toy)